In [1]:
import numpy as np
import tensorflow as tf 
from collections import deque
import random
import pdb
from replay_memory_M import ReplayMemory # AC replay memory
from AC_networks_tf import ActorNet, CriticNet
from test_game import test_env
import os
from __future__ import print_function
from info_obj import info_holder
import time

In [2]:
import gym
#game_name = 'LunarLander-v2'
game_name = 'CartPole-v0'
#game_name = 'MountainCar-v0'
game = gym.make(game_name)
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
'''
game = test_env()
state_dim = 1
action_dim = 2
'''

[2017-01-03 19:35:45,540] Making new env: CartPole-v0


'\ngame = test_env()\nstate_dim = 1\naction_dim = 2\n'

In [3]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
if_load = 'n'
act_hid_lay = [32, 32]
crit_hid_lay = [32, 32]
save_dir = './Results/'+ game_name[:-3] + '/net/'
if 'LunarLander' in game_name:
    LEARNING_RATE = float(1e-1) #5e-5 lr/2
    lr_decay_rate = 1
    C_update = 1 #episode
    T_MAX = 8
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = int(8)
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'CartPole' in game_name:
    LEARNING_RATE = 1e-3 #5e-5 lr/2
    lr_decay_rate = 1
    C_update = 1 #episode
    T_MAX = 5
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = int(8)
    GAMMA = 0.99
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
    
elif 'MountainCar' in game_name:
    LEARNING_RATE = 1e-1 #5e-5 lr/2
    lr_decay_rate = 1
    C_update = 1 #episode
    T_MAX = 5
    BUFFER_SIZE = int(1e5)
    MINIBATCH_SIZE = int(8)
    GAMMA = 0.99
    MAX_EPISODE = int(5e4)
    MAX_STEP = int(2e2)
    lr_decay_ep = 100
    print_ep = int(1e2)
    save_ep = int(1e3)
    TAU = 0
    ENTROPY_BETA = 0
#### load the network

logging_mode = 'w'
strings = ['learning_rate = ', 'lr decay = ', 'Buffer size = ', 'Batch size = ',\
           'actor hidden layers: ', 'critic hidden layers: ', 'update cycle: ', 't_max = ']
if 'y' == if_load:
    if not(os.path.isdir(save_dir)):
        raise NameError('No network to upload')
    logging_mode = 'a'
    with open(save_dir + 'params.txt', 'r') as f:
        xa = f.readline()
        while xa:
            if strings[0] in xa:
                LEARNING_RATE = float(xa[len(strings[0]):])
            if strings[1] in xa:
                lr_decay_rate = float(xa[len(strings[1]):])
            elif strings[2] in xa:
                BUFFER_SIZE = int(xa[len(strings[2]):])
            elif strings[3] in xa:
                MINIBATCH_SIZE = int(xa[len(strings[3]):])
            elif strings[4] in xa:
                act_hid_lay = [int(xb) for xb in xa[len(strings[4]):].split()]
            elif strings[5] in xa:
                crit_hid_lay = [int(xb) for xb in xa[len(strings[5]):].split()]
            elif strings[6] in xa:
                C_update = int(xa[len(strings[6]):])
            elif strings[7] in xa:
                T_MAX = int(xa[len(strings[7]):])
            xa = f.readline()
    with open(save_dir + 'rewards.txt', 'r') as f:
        num_lines = sum(1 for line in f)
        start_ep = num_lines - 1
    var = raw_input("From what episode?(Enter a number or last): ")
    if var != 'last': 
        var_int = int(var)
        while ((var_int%save_ep)!=0) or (var_int > start_ep):
            print('Episode not found!')
            var = raw_input("From what episode?(Enter a number or last): ")
    print('Loading the previous trained network and continue training from episode %d' %start_ep)
else:
    start_ep = 0
lr = LEARNING_RATE * (lr_decay_rate ** (start_ep/lr_decay_ep))# / MINIBATCH_SIZE
#### till here

buff = ReplayMemory(BUFFER_SIZE, T_MAX, GAMMA)
info = info_holder()
try:
    actor
except:
    pass
else:
    actor.reset_net()
try:
    critic
except:
    pass
else:
    critic.reset_net()  
    
graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.act_hid_layers = np.array(act_hid_lay)
    info.crit_hid_layers = np.array(crit_hid_lay)
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.entropy_beta = ENTROPY_BETA
    actor = ActorNet(info)
    critic = CriticNet(info)

In [4]:
# lr = LEARNING_RATE
buff.clear()
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.global_variables_initializer())
    
    if if_load == 'y':
        saver = tf.train.Saver()
        filename = save_dir + 'Episod_' + str(start_ep-1) + '.chk'
        saver.restore(sess, filename)
        buff.load_buf(save_dir)
        
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    ep_var = np.zeros(MAX_EPISODE)
    buffer_time = np.zeros(MAX_EPISODE)
    train_time = np.zeros(MAX_EPISODE)
    avg_var = 0
    
    for ep in range(MAX_EPISODE):

        s = game.reset()
        var = 0
        for st in range(MAX_STEP):
            a = actor.take_action(s.reshape(-1,info.dim_s))
            s2, r, terminal, report = game.step(a)

            buff.add(np.reshape(s, (info.dim_s,)).astype(np.float32), np.array(a).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (info.dim_s,)).astype(np.float32),\
                     np.array(terminal).astype(np.float32))
            if (buff.size() - T_MAX) >= MINIBATCH_SIZE:
                #start = time.time()
                s2_batch, x1, x2 = buff.sample_batch(MINIBATCH_SIZE)
                buffer_time[ep] += x1
                train_time[ep] += x2
                #buffer_time[ep] += (time.time() - start)
                s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
                V_2 = critic.predict(s2_batch)
                V_2 = V_2[:,0]
                start = time.time()
                (s_batch, a_batch, R_batch) = buff.nstep_calculation(V_2)
                s_batch = np.array(s_batch).reshape(-1, info.dim_s)
                a_batch = np.array(a_batch)#.reshape(-1, 1)
                R_batch = np.array(R_batch).reshape(-1, 1)
                #train_time[ep] += (time.time() - start)
                
                predicted_Q, _, _ = critic.train(s_batch, R_batch, lr)
                td_batch = R_batch - predicted_Q
                actor.train(s_batch, a_batch, td_batch, lr)
                
                
                if(ep == 300) and (st == 1):
                    pdb.set_trace()
            ep_reward[ep] += (1.*r)
            var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2

            if terminal:
                ep_var[ep] = var/st
                buffer_time[ep] = buffer_time[ep]/st
                train_time[ep] = train_time[ep]/st
                #avg_var += (var/print_ep)
                break
        '''
        if ((ep % C_update) == (C_update -1)):
            (s_batch, a_batch, r_batch, s2_batch, t_batch) = \
                buff.sample_batch()
            s_batch = np.array(s_batch).reshape(-1, info.dim_s)
            s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
            a_batch = np.array(a_batch)#.reshape(-1, 1)
            r_batch = np.array(r_batch)#.reshape(-1, 1)
            t_batch = np.array(t_batch)#.reshape(-1, 1)

            Q_2 = critic.predict(s2_batch)
            Q_2 = Q_2[:,0]
            
            R_batch = r_batch + GAMMA * Q_2 * (1-t_batch)
            R_batch = R_batch.reshape(-1, 1)
            #pdb.set_trace()
            predicted_Q, _, _ = critic.train(s_batch, R_batch, lr)
            td_batch = R_batch - predicted_Q
            actor.train(s_batch, a_batch, td_batch, lr)
        '''
        if terminal == False:
            ep_var[ep] = var/MAX_STEP
            buffer_time[ep] = buffer_time[ep]/MAX_STEP
            train_time[ep] = train_time[ep]/MAX_STEP

        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Ep: %d --- Ave R: %.1f  |  Suc: %.2f  |  crash: %.2f  |  Max R: %.1f  |  var: %.2f  |  lr:%.2fe-4  |  buffer_time: %f  |  train_time: %f'\
                  %((start_ep+ep), np.mean(ep_reward[last_ep:ep+1]), np.mean(ep_success[last_ep:ep+1]), \
                    np.mean(ep_crash[last_ep:ep+1]), np.amax(ep_reward[last_ep:ep+1]), np.mean(ep_var[last_ep:ep+1]), \
                    1e4*lr, np.mean(buffer_time[last_ep:ep+1]), np.mean(train_time[last_ep:ep+1])))
            last_ep = ep
            #avg_var = 0
        
        if (ep == 0) and (if_load == 'n'):
            if (os.path.isdir(save_dir)):
                raise NameError('Directory net exists')
                
        if (ep == (save_ep-1) and (if_load == 'n')):
            os.mkdir(save_dir)
            with open(save_dir + 'params.txt', 'w') as f:
                f.write('Learning rate = %f\n' %LEARNING_RATE)
                f.write('lr decay = %f\n' %lr_decay_rate)
                f.write('Buffer size = %d\n' %BUFFER_SIZE)
                f.write('Batch size = %d\n' %MINIBATCH_SIZE)
                f.write('t_max = %d\n' %T_MAX)
                f.write('actor hidden layers: %s\n' %str(info.act_hid_layers)[1:-1])
                f.write('critic hidden layers: %s\n' %str(info.crit_hid_layers)[1:-1])
                f.write('update cycle: %d\n' %C_update)
                
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ...' %(start_ep+ep), end="")
            saver = tf.train.Saver()
            filename = save_dir + 'Episod_' + str(start_ep+ep) + '.chk'
            saver.save(sess, filename)
            if (ep == (save_ep-1)) and (if_load != 'y'):
                with open(save_dir + 'rewards.txt', 'w') as f:
                    f.write('x, , , , ,Learning rate = %f - lr decay = %f - C_update = %d - hidden layers: %d\n' \
                            %(LEARNING_RATE, lr_decay_rate, C_update, info.hid_layers) )
            with open(save_dir + 'rewards.txt', 'a') as f:     
                for i in range(ep-save_ep+1, ep+1):
                    f.write('%f,     %f\n' %(ep_reward[i],ep_var[i]))
                buff.save_buf(save_dir)
            print(' Done ******')
       
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = lr_decay_rate * lr
    
print('DONE') 
    

Ep: 0 --- Ave R: 15.0  |  Suc: 0.00  |  crash: 0.00  |  Max R: 15.0  |  var: 0.54  |  lr:10.00e-4  |  buffer_time: 0.000000  |  train_time: 0.000000
Ep: 99 --- Ave R: 20.9  |  Suc: 0.00  |  crash: 0.00  |  Max R: 67.0  |  var: 0.54  |  lr:10.00e-4  |  buffer_time: 0.000014  |  train_time: 0.000497
Ep: 199 --- Ave R: 21.9  |  Suc: 0.00  |  crash: 0.00  |  Max R: 64.0  |  var: 0.55  |  lr:10.00e-4  |  buffer_time: 0.000012  |  train_time: 0.001354
Ep: 299 --- Ave R: 21.1  |  Suc: 0.00  |  crash: 0.00  |  Max R: 57.0  |  var: 0.55  |  lr:10.00e-4  |  buffer_time: 0.000017  |  train_time: 0.002201
> <ipython-input-4-fa37a6904492>(56)<module>()
-> ep_reward[ep] += (1.*r)
(Pdb) c
Ep: 399 --- Ave R: 35.4  |  Suc: 0.00  |  crash: 0.00  |  Max R: 77.0  |  var: 0.56  |  lr:10.00e-4  |  buffer_time: 0.000015  |  train_time: 0.003249
Ep: 499 --- Ave R: 58.7  |  Suc: 0.00  |  crash: 0.00  |  Max R: 122.0  |  var: 0.66  |  lr:10.00e-4  |  buffer_time: 0.000014  |  train_time: 0.005346
Ep: 599 --- Av

BdbQuit: 

## 